In [ ]:
from ultralytics import YOLO
from nn import LCBHAM
from utils import product_dict, stringify_map, init_dsp_yolo
from copy import deepcopy
import pandas as pd

In [2]:
def lcbham_conds(inject, multi_lcam, multi_ldsam, shared):
    return (
        (inject != 0 and (
            (multi_lcam == False and multi_ldsam == False) or # Smallest number of parameters
            (multi_lcam == True and multi_ldsam == False) or # Paper implementation
            (multi_lcam == True and multi_ldsam == True) # Largest number of parameters
        )) or (inject == 0 and multi_lcam == False and multi_ldsam == False and shared == False)  # Plain YOLOv8n, no inject, no multi
    )

In [3]:
global_params = {
    "inject": [0, 1, 2],
    "multi_lcam": [False, True],
    "multi_ldsam": [False, True],
    "shared": [False, True],
}
# multi_lcam = True and multi_ldsam = False is the paper implementation

In [4]:
common_train_params = {
    "data": "dataset.yaml",
    "epochs": 200,      # High enough to allow full learning
    "patience": 30,     # Stop if no improvement for 30 epochs
    "imgsz": 640,       # Standard for YOLOv8/v9/v10/v11
    "batch": -1,        # AutoBatch: Adjust based on your GPU VRAM st 60% in use
    "save": True,        # Ensures weights are saved
    "save_period": 20,     # Save every 20 epochs for later analysis
    #"cache": True,       # Cache dataset in RAM for faster training
    "project": "dsp-yolo-results",
    "resume": True,         # Resume training if interrupted
    "profile": True,        # Enable training profiling for performance insights
}

In [ ]:
NUM_RUNS = 3

In [ ]:
for param in product_dict(global_params, lcbham_conds):
    results_list = []
    for i in range(NUM_RUNS):
        inject = param.pop("inject")  # inject is not a parameter for LCBHAM, it's just a flag for whether to inject or not
        model, train_layers_list = init_dsp_yolo(inject=inject, **param)

        print("Plain YOLOv8n" if inject == 0 else stringify_map(param, "="))
        model.info(detailed=True, imgsz=common_train_params["imgsz"])
        freeze_layers_list = None
        if train_layers_list != None:
            name1 = stringify_map({**param, "step": 1})
            # Freeze all layers except those in train_layers_list
            freeze_layers_list = [k for k in range(1, 23) if k not in model.train_layers_list]
            model.train(name=name1, freeze=freeze_layers_list, lr0=0.01, **common_train_params)

        model, _ = init_dsp_yolo(inject=inject, **param)
        name2 = None
        if inject == 0:
            name2 = "yolov8n_plain"
        else:
            name2 = stringify_map({**param, "step": 2})
            model.load(model.trainer.best)
        results = model.train(name=name2, freeze=freeze_layers_list, lr0=0.001, **common_train_params)
        results_list.append(results.results_dict)
    df = pd.DataFrame(results_list)
    average_metrics = df.mean().to_frame(name='Average_Value')
    average_metrics.to_csv(f"{stringify_map(param)}_average_metrics.csv")

Plain YOLOv8n
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]  -0.00279     0.152        float32
    1                       model.0.bn.weight         BatchNorm2d     False          16                [16]      2.97      1.86        float32
    1                         model.0.bn.bias         BatchNorm2d     False          16                [16]     0.249      4.17        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  -0.00012     0.063        float32
    4                       model.1.bn.weight         BatchNorm2d     False          32                [32]      5.02      1.12        float32


RuntimeError: Dataset 'dataset.yaml' error ❌ 'dataset.yaml' does not exist